In [4]:
import datetime
import xarray  as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [5]:
def open_multiple_nc(nc_files):
    ds = xr.open_mfdataset(nc_files, concat_dim='nRec', combine='nested', parallel=True)

    #ds = ds.sel(nRec=slice(16500, 17000))

    return ds

dir = '/home/sachin/Documents/NIPR/Research/Data/AMPERE/raw/2017/*.nc'
open_multi_amp = open_multiple_nc(dir)
open_multi_amp


<xarray.Dataset> Size: 65GB
Dimensions:       (nRec: 259358, nObs: 1200, vComp: 3)
Dimensions without coordinates: nRec, nObs, vComp
Data variables: (12/34)
    npnt          (nRec) int16 519kB dask.array<chunksize=(720,), meta=np.ndarray>
    year          (nRec) int16 519kB dask.array<chunksize=(720,), meta=np.ndarray>
    doy           (nRec) int16 519kB dask.array<chunksize=(720,), meta=np.ndarray>
    time          (nRec) float32 1MB dask.array<chunksize=(720,), meta=np.ndarray>
    avgint        (nRec) int16 519kB dask.array<chunksize=(720,), meta=np.ndarray>
    kmax          (nRec) int16 519kB dask.array<chunksize=(720,), meta=np.ndarray>
    ...            ...
    del_db_geo    (nRec, nObs, vComp) float64 7GB dask.array<chunksize=(720, 1200, 3), meta=np.ndarray>
    del_jPar      (nRec, nObs) float64 2GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Th  (nRec, nObs) float64 2GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Th  (nRec, nObs) float64 2GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Ph  (nRec, nObs) float64 2GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Ph  (nRec, nObs) float64 2GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
Attributes:
    title:        Ampere data for 2017-01-01 00:00 - 2017-01-02 00:00.
    description:  AMPERE GRD data (fitted magnetic pertubation data).
    created:      2022-09-11T15:27:09Z

In [4]:
def create_datetime(ds):

    ds = ds.copy()
    fractional_hours = ds['time'].values
    year = ds['year'].values
    doy = ds['doy'].values

    df = pd.DataFrame({'fractional_hours': fractional_hours})
    df['hours'] = df['fractional_hours'].apply(lambda x: int(x))
    df['minutes'] = df['fractional_hours'].apply(lambda x: int((x * 60) % 60))
    #round up minutes to nearest 2
    df['minutes'] = df['minutes'].apply(lambda x: x + 1 if x % 2 != 0 else x)

    df['time'] = df.apply(lambda row: f"{int(row['hours']):02}:{int(row['minutes']):02}", axis=1)
    df['doy'] = doy
    df['doy'] = df['doy'].astype(int)
    df['year'] = year
    df['year'] = df['year'].astype(int)

    def doy_year_to_date(year, doy):
        return datetime.datetime(year, 1, 1) + datetime.timedelta(int(doy) - 1)

    df['date'] = df.apply(lambda row: doy_year_to_date(row['year'], row['doy']), axis=1)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['dt'] = pd.to_datetime(df['date'] + ' ' + df['time'])

    #add dt to ds
    ds['dt'] = df['dt']
    ds = ds.sortby('dt')

    return ds

open_amp = create_datetime(open_multi_amp)
open_amp

<xarray.Dataset> Size: 180MB
Dimensions:       (nRec: 720, nObs: 1200, vComp: 3, dim_0: 720)
Coordinates:
  * dim_0         (dim_0) int64 6kB 0 1 2 3 4 5 6 ... 714 715 716 717 718 719
Dimensions without coordinates: nRec, nObs, vComp
Data variables: (12/35)
    npnt          (nRec) int16 1kB dask.array<chunksize=(720,), meta=np.ndarray>
    year          (nRec) int16 1kB dask.array<chunksize=(720,), meta=np.ndarray>
    doy           (nRec) int16 1kB dask.array<chunksize=(720,), meta=np.ndarray>
    time          (nRec) float32 3kB dask.array<chunksize=(720,), meta=np.ndarray>
    avgint        (nRec) int16 1kB dask.array<chunksize=(720,), meta=np.ndarray>
    kmax          (nRec) int16 1kB dask.array<chunksize=(720,), meta=np.ndarray>
    ...            ...
    del_jPar      (nRec, nObs) float64 7MB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Th  (nRec, nObs) float64 7MB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Th  (nRec, nObs) float64 7MB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Ph  (nRec, nObs) float64 7MB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Ph  (nRec, nObs) float64 7MB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    dt            (dim_0) datetime64[ns] 6kB 2013-05-14 ... 2013-05-14T23:58:00
Attributes:
    title:        Ampere data for 2013-05-14 00:00 - 2013-05-15 00:00.
    description:  AMPERE GRD data (fitted magnetic pertubation data).
    created:      2022-09-11T03:06:40-04:00

In [5]:
def create_new_ds(ds):

    ds = ds.copy()
    jPar = ds['jPar'].values
    dt = ds['dt'].values
    mlat = np.linspace(40, 90, 50)
    mlt = np.linspace(0, 24, 24)

    ds_new = xr.Dataset({'jPar': (['dt','mlat','mlt'], jPar.reshape(-1, 50, 24))},
                        coords={'dt': dt,'mlat': mlat,'mlt': mlt})
    
    #round over 5 mins
    ds_new['dt'] = ds_new['dt'].dt.round('5min')
    

    #ds_new = ds_new.sel(dt=slice('2010-08-01 00:00:00', '2010-08-01 23:30:00'))

    return ds_new

ds = create_new_ds(open_amp)
ds

<xarray.Dataset> Size: 7MB
Dimensions:  (dt: 720, mlat: 50, mlt: 24)
Coordinates:
  * dt       (dt) datetime64[ns] 6kB 2013-05-14 ... 2013-05-14T23:58:00
  * mlat     (mlat) float64 400B 40.0 41.02 42.04 43.06 ... 87.96 88.98 90.0
  * mlt      (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 20.87 21.91 22.96 24.0
Data variables:
    jPar     (dt, mlat, mlt) float64 7MB 0.125 0.09338 ... -0.007801 -0.05974

In [6]:
def get_omni(df):
    df = df.copy()
    df['dt'] = df['Timestamp_2min_omni']
    df['dt'] = pd.to_datetime(df['dt'])
    df = df.set_index('dt')
    df = df.drop(columns=['Timestamp_2min_omni','Timestamp_1min_omni','Timestamp_1hr_omni'])

    ds = xr.Dataset.from_dataframe(df)
    ds = ds.sortby('dt')

    return ds

omni_ds = get_omni(pd.read_csv('/home/sachin/Documents/NIPR/Research/Data/OMNI/omni_hro_1min_2013.csv'))
omni_ds

<xarray.Dataset> Size: 26MB
Dimensions:         (dt: 252839)
Coordinates:
  * dt              (dt) datetime64[ns] 2MB 2013-01-01 ... 2013-12-31T23:58:00
Data variables:
    BX_GSE          (dt) float64 2MB 2.42 2.375 2.24 ... -1.105 -1.365 -1.085
    BY_GSE          (dt) float64 2MB -0.365 -0.485 -0.865 ... 4.17 4.185 4.265
    BZ_GSE          (dt) float64 2MB -0.09 -0.105 -0.245 ... -2.6 -2.6 -2.495
    flow_speed      (dt) int64 2MB 358 359 358 361 362 ... 384 383 384 384 384
    proton_density  (dt) float64 2MB 1.94 1.94 1.965 2.065 ... 5.49 5.43 5.64
    AL_INDEX        (dt) float64 2MB -4.5 -6.0 -4.5 -5.5 ... -5.0 -4.0 -4.0 -5.0
    AU_INDEX        (dt) float64 2MB 4.5 3.0 4.0 4.0 3.0 ... 14.5 13.5 11.5 11.0
    SYM_H           (dt) float64 2MB 1.0 1.5 1.5 1.5 1.0 ... 2.0 2.0 2.0 2.5 2.5
    ASY_H           (dt) float64 2MB 12.0 12.5 11.5 12.0 ... 1.5 1.5 2.0 2.5
    F10.7           (dt) int64 2MB 113 113 113 113 113 ... 140 140 140 140 140
    Kp              (dt) float64 2MB 0.7 0.7 0.7 0.7 0.7 ... 0.7 0.7 0.7 0.7 0.7
    doy             (dt) int64 2MB 1 1 1 1 1 1 1 ... 365 365 365 365 365 365 365

In [9]:
#merge omni and ampere data
def merge_omni_ampere(omni_ds, ampere_ds):
    ds = xr.merge([omni_ds, ampere_ds], join='inner')
    #ds = ds.sel(dt=slice('2010-08-01','2010-08-02'))
    #save ds_new to netcdf
    ds = ds.sel(mlat=slice(50, 90))
    ds = ds.sortby('dt')

    ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2013.nc')
    #ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/excluded/20130514.nc')
    
    return ds

merge_omni_ampere(omni_ds, ds)

<xarray.Dataset> Size: 6MB
Dimensions:         (dt: 707, mlat: 40, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 6kB 2013-05-14 ... 2013-05-14T23:58:00
  * mlat            (mlat) float64 320B 50.2 51.22 52.24 ... 87.96 88.98 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Data variables: (12/13)
    BX_GSE          (dt) float64 6kB 3.055 3.24 3.115 ... -6.43 -6.305 -5.815
    BY_GSE          (dt) float64 6kB 2.65 2.71 2.795 2.85 ... 2.825 2.915 2.175
    BZ_GSE          (dt) float64 6kB 2.445 2.535 2.435 2.32 ... 5.235 5.3 5.7
    flow_speed      (dt) int64 6kB 370 369 364 367 370 ... 337 336 333 332 332
    proton_density  (dt) float64 6kB 8.72 8.323 7.13 7.915 ... 9.875 9.965 10.91
    AL_INDEX        (dt) float64 6kB -87.0 -87.5 -87.5 ... -73.0 -73.0 -72.0
    ...              ...
    SYM_H           (dt) float64 6kB 18.0 18.0 18.0 17.0 ... 6.0 5.0 4.5 4.0
    ASY_H           (dt) float64 6kB 31.0 30.0 29.5 29.0 ... 13.0 12.5 12.5 11.5
    F10.7           (dt) int64 6kB 151 151 151 151 151 ... 151 151 151 151 151
    Kp              (dt) float64 6kB 1.3 1.3 1.3 1.3 1.3 ... 1.7 1.7 1.7 1.7 1.7
    doy             (dt) int64 6kB 134 134 134 134 134 ... 134 134 134 134 134
    jPar            (dt, mlat, mlt) float64 5MB 0.03121 -0.02899 ... -0.05974

In [14]:
def nan_count(ds, var):
    #count number of NaNs in a data variable
    con_data = ds[var].values
    nan_count = np.count_nonzero(np.isnan(con_data))
    nan_ratio = nan_count / con_data.size
    not_nan = con_data.size - nan_count
    return not_nan, nan_count, np.round(nan_ratio, 2)

nan_count(merge_omni_ampere(omni_ds, ds), 'jPar')

(297378000, 0, 0.0)

In [15]:
def merge_ds():
    ds_2010 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2010.nc')
    ds_2011 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2011.nc')
    ds_2012 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2012.nc')
    ds_2013 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2013.nc')
    ds_2014 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2014.nc')
    ds_2015 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2015.nc')

    ds = xr.concat([ds_2010, ds_2011, ds_2012, ds_2013, ds_2014, ds_2015], dim='dt')
    ds = ds.sel(mlat=slice(50, 90))
    ds = ds.sortby('dt')

    #print(nan_count(ds, 'jPar'))

    ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2010_2015.nc')

    return ds

merge_ds()

(1420631040, 0, 0.0)


<xarray.Dataset> Size: 12GB
Dimensions:         (dt: 1479824, mlat: 40, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 12MB 2010-01-01 ... 2015-12-31T23:58:00
  * mlat            (mlat) float64 320B 50.2 51.22 52.24 ... 87.96 88.98 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Data variables: (12/13)
    jPar            (dt, mlat, mlt) float64 11GB 0.05463 0.03464 ... -0.05614
    BX_GSE          (dt) float64 12MB 0.075 0.09 0.005 ... 3.89 4.045 3.925
    BY_GSE          (dt) float64 12MB 2.97 3.0 2.965 ... -1.005 -1.12 -1.045
    BZ_GSE          (dt) float64 12MB -0.45 -0.3 -0.545 ... -14.75 -14.66 -14.69
    flow_speed      (dt) int32 6MB 283 283 283 283 285 ... 464 465 462 462 462
    proton_density  (dt) float64 12MB 3.85 3.92 4.02 3.995 ... 2.48 2.565 2.5
    ...              ...
    AU_INDEX        (dt) float64 12MB 3.0 3.0 3.0 3.0 ... 24.0 24.5 37.0 57.0
    SYM_H           (dt) float64 12MB 1.0 1.0 1.0 1.0 ... -98.0 -99.0 -98.5
    ASY_H           (dt) float64 12MB 10.0 10.0 10.5 11.0 ... 72.0 73.0 74.0
    F10.7           (dt) int32 6MB 72 72 72 72 72 72 72 ... 93 93 93 93 93 93 93
    Kp              (dt) float64 12MB 0.0 0.0 0.0 0.0 0.0 ... 5.7 5.7 5.7 5.7
    doy             (dt) int32 6MB 1 1 1 1 1 1 1 ... 365 365 365 365 365 365 365

1480531

1421309760

In [ ]:
fac = ds['jPar'].mean(dim='dt').values

def dt_to_str(dt):
    dt_1 = dt[0]
    dt_2 = dt[1] 
    time_1 = pd.to_datetime(dt_1)
    time_2 = pd.to_datetime(dt_2)
    str_time_1 = time_1.strftime('%Y-%m-%d %H:%M')
    str_time_2 = time_2.strftime('%Y-%m-%d %H:%M')

    return str_time_1, str_time_2

dt = open_amp['dt'].values
start_time, end_time = dt_to_str(dt)

fac = fac.reshape(24, 50).T # reshape and transpose
fac = np.flipud(fac) # flip the array upside down
theta = np.linspace(0, 360, 24) - 90 # rotate by 90 degrees
theta = np.radians(theta) # convert to radians
r = 90 - np.linspace(40, 90, 50) #convert to colat

cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", ["blue","blue", "white", "white","red","red"])
cmap = 'bwr'

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6, 5))

c = ax.contourf(theta, r, fac, cmap=cmap, levels=np.linspace(-0.5,.5 , 100))
ax.set_ylim([0,40])
ax.set_yticks([0, 10, 20, 30,40])
ax.set_yticklabels(["90°", "80°", "70°", "60°","50° MLAT"])
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["21", "0 MLT \nMidnight", "3", "6 \n  Dawn", "9", "12 MLT \nMidday", "15", "18 \nDusk"])
ax.grid(True, linestyle='-.', alpha=0.7)
ax.set_title(f"{start_time} - {end_time}", pad=10, fontsize=11.5)


plt.colorbar(c, ax=ax, label='J$_\parallel$ (FAC) [µA/m$^2$]', shrink=0.3, pad = 0.12, 
             ticks=[-0.5, 0,  0.5], 
             orientation='horizontal')

plt.tight_layout()
#plt.savefig('ampere_20100807.png', dpi=400)